In [1]:
import cv2 as cv
import numpy as np

In [2]:
img = cv.imread("redN.jpg")

In [3]:
img_red = img[:, :, 2]
blur = cv.medianBlur(img_red, 3)

In [4]:
cv.imwrite('redN_blur.jpg', blur)

True

In [1]:
_, redN_bin = cv.threshold(img_red, 230, 255, cv.THRESH_BINARY)

NameError: name 'cv' is not defined

In [6]:
cv.imwrite('redN_bin.jpg', redN_bin)


True

In [7]:
kernel = np.ones((3, 3), np.int8)
red_dil = cv.dilate(redN_bin, kernel, iterations=1)

In [8]:
cv.imwrite("redN_dil.jpg", red_dil)

True

In [9]:
contours, hierarchy = cv.findContours(red_dil, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

In [10]:
hierarchy

array([[[ 1, -1, -1, -1],
        [ 2,  0, -1, -1],
        [ 3,  1, -1, -1],
        [ 4,  2, -1, -1],
        [ 5,  3, -1, -1],
        [-1,  4, -1, -1]]], dtype=int32)

In [11]:
boundRect = []
for c in contours:
    x, y, w, h = cv.boundingRect(c)
    # 一个筛选，可能需要看识别条件而定，有待优化
    if h/w > 1:
        boundRect.append([x,y,w,h])
        # 画一个方形标注一下，看看圈的范围是否正确
        # red_dil = cv.rectangle(red_dil, (x, y), (x+w, y+h), 255, 2)


In [12]:
boundRect

[[208, 53, 64, 94], [51, 53, 61, 84], [130, 50, 65, 94]]

In [13]:
cv.imwrite('redN_rec.jpg', red_dil)

True

In [2]:
# 尝试定义一个特殊方法拓展sorted的适用范围
# 已验证可用
def __lt__(self, y):
    if self[0] < y[0]:
        return True
    else:
        return False

bounds = sorted(boundRect)

# bound

NameError: name 'boundRect' is not defined

In [15]:
def numDetect(img, loc):
    x,y,w,h = loc
    
    if h/w > 3:
        return 1
    else:
        a, b, c, d, e, f, g = 0,0,0,0,0,0,0
        
        # 更新，改为穿针法
        line1 = red_dil[y:y+h, x+w//2]
        line2 = red_dil[y+h//4, x:x+w]
        line3 = red_dil[y+(h//4)*3, x:x+w]
        # 检测竖线，从而识别a,g,d笔画
        a,b,c,d,e,f,g = 0,0,0,0,0,0,0
        for i in range(h):
            if line1[i]==255:
                if i<(h//3):
                    a = 1
                if i>2*(h//3):
                    d = 1
                if i>(h//3) and i<2*(h//3):
                    g = 1
        # 检测横线line2、line3，从而识别b,f笔画并减少时间消耗
        for i in range(w):
            if line2[i]==255:
                if i<(w//2):
                    b = 1
                if i>(w//2):
                    f = 1
            if line3[i]==255:
                if i<(w//2):
                    c = 1
                if i>(w//2):
                    e = 1
        
        # 不写的眼花缭乱了，直接写就可以了
        if a and b and c and d and e and f and g==0:
            return 0
        if a and b==0 and c and d and e == 0 and f and g:
            return 2
        if a and b==0 and c==0 and d and e and f and g:
            return 3
        if a==0 and b and c==0 and d==0 and e and f and g:
            return 4
        if a and b and c==0 and d and e and f==0 and g:
            return 5
        if a and b and c and d and e and f==0 and g:
            return 6
        if a and b==0 and c==0 and d==0 and e and f and g==0:
            return 7
        if a and b and c and d and e and f and g:
            return 8
        if a and b and c==0 and d and e and f and g:
            return 9
        
        return -1



In [16]:
num = 0
for i in bounds:
    num = num*10 + numDetect(red_dil, i)
    
num

496

In [17]:
# 小数点识别，数字是否为斜着的好像也不干扰

In [18]:
def detectComma(img, bound):
    for i in range(1, len(bound)):
        
        roi_x1 = bound[i-1][0]+bound[i-1][2]
        roi_x2 = bound[i][0]
        roi_y2 = max(bound[i-1][1]+bound[i-1][3], bound[i][1]+bound[i][3])
        roi_y1 = roi_y2 - max(bound[i-1][3], bound[i][3])//3
        roi = img[roi_y1:roi_y2, roi_x1:roi_x2]
        
        contours0, hierarchy0 = cv.findContours(roi, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
        
        boundRect0 = []
        for c in contours0:
            x, y, w, h = cv.boundingRect(c)
            # 一个筛选，可能需要看识别条件而定，有待优化——比如增加小数点的大小判断
            if h/w > 0.75 and h/w < 1.25 :
                boundRect0.append([x,y,w,h])
                # 画一个方形标注一下，看看圈的范围是否正确
                red_dil = cv.rectangle(roi, (x, y), (x+w, y+h), 255, 2)
                
                return i
        
    return -1


In [19]:
detectComma(red_dil, bounds)

-1

In [20]:
# 斜体数字的小数点识别，在数字区域内进行boundRect识别

In [21]:
def detectCommaItalic(img, bounds):
    for i in range(len(bounds)):
        x,y,w,h = bounds[i]
        roi = img[y+3*h//4: y+h, x+2*w//3: x+w]
        
        contours0, hierarchy0 = cv.findContours(roi, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
        
        boundRect0 = []
        for c in contours0:
            x, y, w, h = cv.boundingRect(c)
            # 一个筛选，可能需要看识别条件而定，有待优化——比如增加小数点的大小判断
            if h/w > 0.8 and h/w < 1.2 :
                boundRect0.append([x,y,w,h])
                # 画一个方形标注一下，看看圈的范围是否正确
                red_dil = cv.rectangle(roi, (x, y), (x+w, y+h), 255, 2)
                
                return i
        
    return -1


In [22]:
detectCommaItalic(red_dil, bounds)

1

In [23]:
k = detectComma(red_dil, bounds)

if k != -1:
    # 正体数字检测到小数点的情况
    num = num / pow(10, len(bounds)-a)
else:
    # 正体无小数点或斜体数字的情况
    a = detectCommaItalic(red_dil, bounds)
    
    if a != -1:
        # 斜体数字有小数点的情况
        num = num / pow(10, len(bounds)-1-a)
num

49.6